In [ ]:
import os
import numpy as np
import pandas as pd

from pathlib import Path
from glob import glob
from dotenv import load_dotenv

In [10]:
res_dir = r'c:/Users/by003457/downloads'
rev_file = 'company_revenue_analysis_strategy01.xlsx'

In [25]:
res_dirs = [Path(x) for x in glob(os.path.join(res_dir, 'strategy01_*'))]
res_dirs

[WindowsPath('c:/Users/by003457/downloads/strategy01_aggressive'),
 WindowsPath('c:/Users/by003457/downloads/strategy01_conservative'),
 WindowsPath('c:/Users/by003457/downloads/strategy01_creative')]

In [ ]:
res_datasets = []
for rd in res_dirs:
    file_path = rd / rev_file
    if not file_path.exists():
        continue
    df = pd.read_excel(file_path)
    df['result_dir'] = rd.name
    cols = df.columns.tolist()
    cols = ['result_dir'] + [col for col in cols if col != 'result_dir']
    df = df[cols]
    res_datasets.append(df)
if not res_datasets:
    raise FileNotFoundError("No result directories contained the revenue analysis file.")

In [ ]:
res_datasets[0].head(2)

,result_dir,revenue,tickerSymbol,companyName,indu_desc,desc_1,desc_2,desc_3,desc_4,desc_5,p_neg20below,p_neg5to-20,p_neg5to5,p_5to20,p_above20,total_trades,p_above20_ratio
0,strategy01_aggressive,24063390,A000760,"Rifa Co.,Ltd.",Chemical Manufacturing,Manufacturing,Chemical Manufacturing,NaN,NaN,NaN,0,0,0,0,0,0,0
1,strategy01_aggressive,16400865,A004090,Korea Petroleum Industries Company,"Asphalt Paving, Roofing, and Saturated Materia...",Manufacturing,Petroleum and Coal Products Manufacturing,Petroleum and Coal Products Manufacturing,"Asphalt Paving, Roofing, and Saturated Materia...",NaN,0,0,0,0,0,0,0


In [ ]:
combined_df = pd.concat(res_datasets, ignore_index=True)
combined_df.head(2)

,result_dir,revenue,tickerSymbol,companyName,indu_desc,desc_1,desc_2,desc_3,desc_4,desc_5,p_neg20below,p_neg5to-20,p_neg5to5,p_5to20,p_above20,total_trades,p_above20_ratio
0,strategy01_aggressive,24063390,A000760,"Rifa Co.,Ltd.",Chemical Manufacturing,Manufacturing,Chemical Manufacturing,NaN,NaN,NaN,0,0,0,0,0,0,0.0
1,strategy01_aggressive,16400865,A004090,Korea Petroleum Industries Company,"Asphalt Paving, Roofing, and Saturated Materia...",Manufacturing,Petroleum and Coal Products Manufacturing,Petroleum and Coal Products Manufacturing,"Asphalt Paving, Roofing, and Saturated Materia...",NaN,0,0,0,0,0,0,0.0


In [ ]:
expected_profit_bins = ["p_neg20below", "p_neg5to-20", "p_neg5to5", "p_5to20", "p_above20"]
for col in expected_profit_bins:
    if col not in combined_df.columns:
        combined_df[col] = 0
if "total_trades" not in combined_df.columns:
    combined_df["total_trades"] = combined_df[expected_profit_bins].sum(axis=1)
combined_df["total_trades"] = combined_df["total_trades"].fillna(0).astype(float)
if "p_above20_ratio" not in combined_df.columns:
    combined_df["p_above20_ratio"] = np.where(
        combined_df["total_trades"] > 0,
        combined_df["p_above20"] / combined_df["total_trades"],
        np.nan,
    )
combined_df.head(2)

In [ ]:
profit_bin_cols = ["p_neg20below", "p_neg5to-20", "p_neg5to5", "p_5to20", "p_above20"]
ticker_column = None
if "tickerSymbol" in combined_df.columns:
    ticker_column = "tickerSymbol"
elif "ticker" in combined_df.columns:
    ticker_column = "ticker"
agg_dict = {'revenue': 'sum', 'total_trades': 'sum'}
if ticker_column:
    agg_dict[ticker_column] = 'nunique'
agg_base = combined_df.groupby('result_dir').agg(agg_dict).reset_index()
agg_base = agg_base.rename(columns={'revenue': 'total_revenue'})
if ticker_column and ticker_column in agg_base.columns:
    agg_base = agg_base.rename(columns={ticker_column: 'unique_tickers'})
else:
    agg_base['unique_tickers'] = np.nan
bin_totals = combined_df.groupby('result_dir')[profit_bin_cols].sum().reset_index()
summary_by_result = agg_base.merge(bin_totals, on='result_dir', how='left')
summary_by_result['total_trades'] = summary_by_result['total_trades'].astype(float)
summary_by_result = summary_by_result.assign(
    avg_revenue_per_trade=lambda df_: np.where(
        df_['total_trades'] > 0,
        df_['total_revenue'] / df_['total_trades'],
        np.nan,
    ),
    win_trades=lambda df_: df_[['p_5to20', 'p_above20']].sum(axis=1),
    nearflat_trades=lambda df_: df_['p_neg5to5'],
    loss_trades=lambda df_: df_[['p_neg20below', 'p_neg5to-20']].sum(axis=1),
    win_ratio=lambda df_: np.where(
        df_['total_trades'] > 0,
        df_['win_trades'] / df_['total_trades'],
        np.nan,
    ),
    high_win_ratio=lambda df_: np.where(
        df_['total_trades'] > 0,
        df_['p_above20'] / df_['total_trades'],
        np.nan,
    ),
    estimated_avg_profit_pct=lambda df_: np.where(
        df_['total_trades'] > 0,
        100
        * (
            df_['p_neg20below'] * (-0.25)
            + df_['p_neg5to-20'] * (-0.125)
            + df_['p_neg5to5'] * 0.0
            + df_['p_5to20'] * 0.125
            + df_['p_above20'] * 0.25
        )
        / df_['total_trades'],
        np.nan,
    ),
)
summary_by_result = summary_by_result.round({
    'avg_revenue_per_trade': 2,
    'win_ratio': 3,
    'high_win_ratio': 3,
    'estimated_avg_profit_pct': 2,
})
ordered_columns = [
    'result_dir',
    'unique_tickers',
    'total_trades',
    'total_revenue',
    'avg_revenue_per_trade',
    'estimated_avg_profit_pct',
    'win_trades',
    'loss_trades',
    'nearflat_trades',
    'win_ratio',
    'high_win_ratio',
    'p_neg20below',
    'p_neg5to-20',
    'p_neg5to5',
    'p_5to20',
    'p_above20',
 ]
summary_by_result[ordered_columns]

,result_dir,revenue,total_trades
0,strategy01_aggressive,461782669,0
1,strategy01_conservative,1305402,7649
2,strategy01_creative,96696067,74018


In [ ]:
consistent_profit_metrics = summary_by_result[[
    'result_dir',
    'win_trades',
    'loss_trades',
    'nearflat_trades',
    'total_trades',
    'win_ratio',
    'high_win_ratio',
    'estimated_avg_profit_pct',
]].copy()
consistent_profit_metrics

In [ ]:
profit_bin_distribution = summary_by_result[[
    'result_dir',
    'p_neg20below',
    'p_neg5to-20',
    'p_neg5to5',
    'p_5to20',
    'p_above20',
    'total_trades',
]].copy()
for col in ['p_neg20below', 'p_neg5to-20', 'p_neg5to5', 'p_5to20', 'p_above20']:
    profit_bin_distribution[f'pct_{col}'] = np.where(
        profit_bin_distribution['total_trades'] > 0,
        profit_bin_distribution[col] / profit_bin_distribution['total_trades'],
        np.nan,
    )
profit_bin_distribution

In [ ]:
top_ticker_by_revenue = (
    combined_df
    .groupby(['result_dir', 'tickerSymbol', 'companyName'], dropna=False)
    .agg({
        'revenue': 'sum',
        'total_trades': 'sum',
        'p_neg20below': 'sum',
        'p_neg5to-20': 'sum',
        'p_neg5to5': 'sum',
        'p_5to20': 'sum',
        'p_above20': 'sum',
    })
    .rename(columns={'revenue': 'total_revenue'})
    .reset_index()
 )
top_ticker_by_revenue = top_ticker_by_revenue.assign(
    win_trades=lambda df_: df_[['p_5to20', 'p_above20']].sum(axis=1),
    high_win_ratio=lambda df_: np.where(
        df_['total_trades'] > 0,
        df_['p_above20'] / df_['total_trades'],
        np.nan,
    ),
    win_ratio=lambda df_: np.where(
        df_['total_trades'] > 0,
        df_['win_trades'] / df_['total_trades'],
        np.nan,
    ),
)
top_ticker_by_revenue.sort_values(['result_dir', 'total_revenue'], ascending=[True, False]).groupby('result_dir').head(5)